In [76]:
import pandas as pd

In [77]:
from bs4 import BeautifulSoup as bs

In [78]:
from urllib.request import urlopen as uReq

In [79]:
import os
from urllib.error import HTTPError

In [80]:
df = pd.read_csv('Input.csv')

In [81]:
df.dropna(inplace=True)

In [82]:
df

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...
...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...


In [83]:
uClient  = uReq('https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/')
page = uClient.read()
html = bs(page,"html.parser")
title = html.findAll("h1",{"class":"entry-title"})
content = html.findAll("div",{"class":"td-post-content"})

In [84]:
title = title[0].get_text()
title

'Rise of e-health and its impact on humans by the year 2030'

In [85]:
str = content[0].get_text()
content = str.replace("\n","")
content

'The rise of e-health, or the use of electronic means to facilitate health care, has been a major development in the healthcare industry in recent years. The use of technology to improve access to healthcare and make it more efficient has the potential to revolutionize the way we think about healthcare and its delivery. By 2030, it is likely that e-health will have a significant impact on the way we receive and provide healthcare, and this essay will explore the potential impacts of e-health on humans by that year.One of the main benefits of e-health is the increased access it provides to healthcare services. With the use of telemedicine and other remote care technologies, patients can receive care from anywhere with an internet connection. This can be particularly beneficial for those who live in rural or underserved areas, where access to healthcare providers may be limited. It can also be useful for those with mobility issues or disabilities that make it difficult for them to visit 

In [86]:
title + " " + content

'Rise of e-health and its impact on humans by the year 2030 The rise of e-health, or the use of electronic means to facilitate health care, has been a major development in the healthcare industry in recent years. The use of technology to improve access to healthcare and make it more efficient has the potential to revolutionize the way we think about healthcare and its delivery. By 2030, it is likely that e-health will have a significant impact on the way we receive and provide healthcare, and this essay will explore the potential impacts of e-health on humans by that year.One of the main benefits of e-health is the increased access it provides to healthcare services. With the use of telemedicine and other remote care technologies, patients can receive care from anywhere with an internet connection. This can be particularly beneficial for those who live in rural or underserved areas, where access to healthcare providers may be limited. It can also be useful for those with mobility issue

In [87]:
output_folder = 'title_content'
os.makedirs(output_folder, exist_ok=True)
texts = []
for index, row in df.iterrows():
    try:
        url_id = row['URL_ID']
        url = row['URL']
        uClient  = uReq(url)
        page = uClient.read()
        html = bs(page,"html.parser")
        title = html.findAll("h1",{"class":"entry-title"})
        content = html.findAll("div",{"class":"td-post-content"})
        if title==[]:
            title = html.findAll("h1",{"class":"tdb-title-text"})
        if content==[]:
            content = html.findAll("div",{"class":"tdb-block-inner"})
        title = title[0].get_text().lower()
        content = content[0].get_text().replace("\n","").lower()
        texts.append(title + " " + content)
        file_path = os.path.join(output_folder, f'{url_id}.txt')

        with open(file_path, "w", encoding='utf-8') as f:
            f.write(title + " " + content)

    except HTTPError as e:
        df.drop(index,inplace=True)
        continue
df['raw_text'] = texts
    
    

In [88]:
df

,URL_ID,URL,raw_text
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...
...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...


In [89]:
import chardet
import codecs

stopwords = []

for f in os.listdir(os.path.join(os.getcwd(), 'StopWords')):
    with open(os.path.join(os.getcwd(), 'StopWords', f), 'rb') as file:
        rawdata = file.read()
        result = chardet.detect(rawdata)
        encoding = result['encoding']
        
    with codecs.open(os.path.join(os.getcwd(), 'StopWords', f), 'r', encoding) as file:
        file_contents = file.read()
        stopwords.append(file_contents)


In [90]:
w = []
for text in stopwords:
    lines = text.split('\n')
    processed_lines = [line.split('|')[0].strip() for line in lines]
    for l in processed_lines:
        if l!='':
            w.append(l.lower())
stopwords = w

In [91]:
stopwords

['ernst',
 'young',
 'deloitte',
 'touche',
 'kpmg',
 'pricewaterhousecoopers',
 'pricewaterhouse',
 'coopers',
 'afghani',
 'ariary',
 'baht',
 'balboa',
 'birr',
 'bolivar',
 'boliviano',
 'cedi',
 'colon',
 'córdoba',
 'dalasi',
 'denar',
 'dinar',
 'dirham',
 'dobra',
 'dong',
 'dram',
 'escudo',
 'euro',
 'florin',
 'forint',
 'gourde',
 'guarani',
 'gulden',
 'hryvnia',
 'kina',
 'kip',
 'konvertibilna marka',
 'koruna',
 'krona',
 'krone',
 'kroon',
 'kuna',
 'kwacha',
 'kwanza',
 'kyat',
 'lari',
 'lats',
 'lek',
 'lempira',
 'leone',
 'leu',
 'lev',
 'lilangeni',
 'lira',
 'litas',
 'loti',
 'manat',
 'metical',
 'naira',
 'nakfa',
 'new lira',
 'new sheqel',
 'ngultrum',
 'nuevo sol',
 'ouguiya',
 'pataca',
 'peso',
 'pound',
 'pula',
 'quetzal',
 'rand',
 'real',
 'renminbi',
 'rial',
 'riel',
 'ringgit',
 'riyal',
 'ruble',
 'rufiyaa',
 'rupee',
 'rupee',
 'rupiah',
 'shilling',
 'som',
 'somoni',
 'special drawing rights',
 'taka',
 'tala',
 'tenge',
 'tugrik',
 'vatu',
 '

In [92]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

In [93]:
df['text'] = df['raw_text'].apply(remove_stopwords)

In [94]:
df

,URL_ID,URL,raw_text,text
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...
...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...


In [95]:
df.to_csv('preprocessed_input.csv')

In [96]:
neg_pos = {}
word = []

for f in os.listdir(os.path.join(os.getcwd(), 'MasterDictionary')):
    with open(os.path.join(os.getcwd(), 'MasterDictionary', f), 'rb') as file:
        rawdata = file.read()
        result = chardet.detect(rawdata)
        encoding = result['encoding']
        
    with codecs.open(os.path.join(os.getcwd(), 'MasterDictionary', f), 'r', encoding) as file:
        file_contents = file.read()
        word.append(file_contents)
    f = f.split('-')
    neg_pos[f[0]] = word[0].split('\n')
    word = []


In [97]:
neg_pos

{'negative': ['2-faced',
  '2-faces',
  'abnormal',
  'abolish',
  'abominable',
  'abominably',
  'abominate',
  'abomination',
  'abort',
  'aborted',
  'aborts',
  'abrade',
  'abrasive',
  'abrupt',
  'abruptly',
  'abscond',
  'absence',
  'absent-minded',
  'absentee',
  'absurd',
  'absurdity',
  'absurdly',
  'absurdness',
  'abuse',
  'abused',
  'abuses',
  'abusive',
  'abysmal',
  'abysmally',
  'abyss',
  'accidental',
  'accost',
  'accursed',
  'accusation',
  'accusations',
  'accuse',
  'accuses',
  'accusing',
  'accusingly',
  'acerbate',
  'acerbic',
  'acerbically',
  'ache',
  'ached',
  'aches',
  'achey',
  'aching',
  'acrid',
  'acridly',
  'acridness',
  'acrimonious',
  'acrimoniously',
  'acrimony',
  'adamant',
  'adamantly',
  'addict',
  'addicted',
  'addicting',
  'addicts',
  'admonish',
  'admonisher',
  'admonishingly',
  'admonishment',
  'admonition',
  'adulterate',
  'adulterated',
  'adulteration',
  'adulterier',
  'adversarial',
  'adversary'

In [98]:
from nltk.tokenize import word_tokenize

In [99]:
df['tokens'] = df['text'].apply(word_tokenize)

In [100]:
def pos_count(token):
    count = 0
    for t in token:
        if t in neg_pos['positive']:
            count=count+1
    return count

In [101]:
def neg_count(token):
    count = 0
    for t in token:
        if t in neg_pos['negative']:
            count=count+1
    return count

In [102]:
df['POSITIVE SCORE'] = 0
df['NEGATIVE SCORE'] = 0
df['POSITIVE SCORE']=df['tokens'].apply(pos_count)
df['NEGATIVE SCORE']=df['tokens'].apply(neg_count)

In [103]:
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27
...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21


In [104]:
df['POLARITY SCORE'] = 0
df['POLARITY SCORE'] = (df['POSITIVE SCORE'] - df['NEGATIVE SCORE'])/ ((df['POSITIVE SCORE'] + df['NEGATIVE SCORE']) + 0.000001)
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032
...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667


In [105]:
df['SUBJECTIVITY SCORE'] = 0
df['SUBJECTIVITY SCORE'] = (df['POSITIVE SCORE'] + df['NEGATIVE SCORE'])/ (len(df['text']) + 0.000001)
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571
...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786


In [115]:
df['AVG SENTENCE LENGTH'] = 0
li = []
for index,row in df.iterrows():
    li.append(len(df['text'][index].split()) / len(df['text'][index].split('.')))
df['AVG SENTENCE LENGTH'] = li
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214,9.551724
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357,11.680000
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786,7.148649
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000
...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786,10.242424
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571,15.980392
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000,12.597222
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786,10.046875


In [158]:
def count_syllables(word):
    vowels = "AEIOUaeiou"
    count = 0
    for char in word:
        if char in vowels:
            count += 1
    exceptions = ["es", "ed"]
    if word[-2:] in exceptions:
        count-=1
    return count

def find_complex_words(words):
    complex_words = [word for word in words if count_syllables(word) > 2]
    return len(complex_words)

In [141]:
df['COMPLEX WORD COUNT'] = 0
df['COMPLEX WORD COUNT'] = df['tokens'].apply(find_complex_words)
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214,9.551724,534
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357,11.680000,194
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786,7.148649,280
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353
...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786,10.242424,150
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571,15.980392,351
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000,12.597222,437
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786,10.046875,337


In [142]:
df['PERCENTAGE OF COMPLEX WORDS'] = 0
df['PERCENTAGE OF COMPLEX WORDS'] = df['COMPLEX WORD COUNT']/len(df['tokens'])
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214,9.551724,534,4.767857
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357,11.680000,194,1.732143
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786,7.148649,280,2.500000
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786
...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786,10.242424,150,1.339286
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571,15.980392,351,3.133929
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000,12.597222,437,3.901786
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786,10.046875,337,3.008929


In [143]:
df['FOG INDEX'] = 0
df['FOG INDEX'] = 0.4*(df['AVG SENTENCE LENGTH']+df['PERCENTAGE OF COMPLEX WORDS'])
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214,9.551724,534,4.767857,5.727833
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357,11.680000,194,1.732143,5.364857
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786,7.148649,280,2.500000,3.859459
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786,10.242424,150,1.339286,4.632684
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571,15.980392,351,3.133929,7.645728
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000,12.597222,437,3.901786,6.599603
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786,10.046875,337,3.008929,5.222321


In [146]:
df['AVG NUMBER OF WORDS PER SENTENCE'] = 0
li = []
for index,row in df.iterrows():
    li.append(len(df['raw_text'][index].split()) / len(df['raw_text'][index].split('.')))
df['AVG NUMBER OF WORDS PER SENTENCE'] = li
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214,9.551724,534,4.767857,5.727833,18.896552
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357,11.680000,194,1.732143,5.364857,24.720000
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786,7.148649,280,2.500000,3.859459,14.689189
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786,10.242424,150,1.339286,4.632684,19.424242
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571,15.980392,351,3.133929,7.645728,31.333333
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000,12.597222,437,3.901786,6.599603,24.027778
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786,10.046875,337,3.008929,5.222321,19.625000


In [150]:
from nltk.corpus import stopwords
def remove_stopword(text):
    new_text = []
    for w in text.split():
        if w not in stopwords.words('english'):
            new_text.append(w)
    return " ".join(new_text)


In [155]:
import string
exclude = string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

In [154]:
df['WORD COUNT'] = 0
li = []
for index,row in df.iterrows():
    li.append(len(remove_punc(remove_stopword(df['raw_text'][index]))))
df['WORD COUNT'] = li
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,WORD COUNT
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214,9.551724,534,4.767857,5.727833,18.896552,8440
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357,11.680000,194,1.732143,5.364857,24.720000,2951
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786,7.148649,280,2.500000,3.859459,14.689189,5114
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068,6256
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068,6256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786,10.242424,150,1.339286,4.632684,19.424242,3366
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571,15.980392,351,3.133929,7.645728,31.333333,8228
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000,12.597222,437,3.901786,6.599603,24.027778,8349
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786,10.046875,337,3.008929,5.222321,19.625000,6397


In [159]:
def count_syllables(word):
    vowels = "AEIOUaeiou"
    count = 0
    for char in word:
        if char in vowels:
            count += 1
    exceptions = ["es", "ed"]
    if word[-2:] in exceptions:
        count-=1
    return count

def syllable_per_word(tokens):
    dict = {}
    for word in tokens:
        dict[word] = count_syllables(word)
    return dict

df['SYLLABLE PER WORD'] = 0
df['SYLLABLE PER WORD'] = df['tokens'].apply(syllable_per_word)
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,WORD COUNT,SYLLABLE PER WORD
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214,9.551724,534,4.767857,5.727833,18.896552,8440,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l..."
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357,11.680000,194,1.732143,5.364857,24.720000,2951,"{'rise': 2, 'e-health': 3, 'impact': 2, 'human..."
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786,7.148649,280,2.500000,3.859459,14.689189,5114,"{'rise': 2, 'e-health': 3, 'impact': 2, 'human..."
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068,6256,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l..."
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068,6256,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786,10.242424,150,1.339286,4.632684,19.424242,3366,"{'coronavirus': 5, ':': 0, 'impact': 2, 'hospi..."
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571,15.980392,351,3.133929,7.645728,31.333333,8228,"{'coronavirus': 5, 'impact': 2, 'energy': 2, '..."
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000,12.597222,437,3.901786,6.599603,24.027778,8349,"{'policies': 3, 'mitigate': 4, 'impacts': 2, '..."
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786,10.046875,337,3.008929,5.222321,19.625000,6397,"{'marketing': 3, 'drives': 1, 'results': 2, 'f..."


In [162]:
import re

def count_personal_pronouns(text):
    pronoun_pattern = r'\b(?:I|we|my|ours|us)\b'
    country_name_pattern = r'\bUS\b'
    pronoun_matches = re.findall(pronoun_pattern, text, flags=re.IGNORECASE)
    country_name_matches = re.findall(country_name_pattern, text)
    personal_pronoun_count = len(pronoun_matches) - len(country_name_matches)
    return personal_pronoun_count

df['PERSONAL PRONOUNS'] = 0
df['PERSONAL PRONOUNS'] = df['raw_text'].apply(count_personal_pronouns)
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214,9.551724,534,4.767857,5.727833,18.896552,8440,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",2
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357,11.680000,194,1.732143,5.364857,24.720000,2951,"{'rise': 2, 'e-health': 3, 'impact': 2, 'human...",3
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786,7.148649,280,2.500000,3.859459,14.689189,5114,"{'rise': 2, 'e-health': 3, 'impact': 2, 'human...",3
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068,6256,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",7
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068,6256,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786,10.242424,150,1.339286,4.632684,19.424242,3366,"{'coronavirus': 5, ':': 0, 'impact': 2, 'hospi...",1
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571,15.980392,351,3.133929,7.645728,31.333333,8228,"{'coronavirus': 5, 'impact': 2, 'energy': 2, '...",3
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000,12.597222,437,3.901786,6.599603,24.027778,8349,"{'policies': 3, 'mitigate': 4, 'impacts': 2, '...",0
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786,10.046875,337,3.008929,5.222321,19.625000,6397,"{'marketing': 3, 'drives': 1, 'results': 2, 'f...",7


In [163]:
def calculate_average_word_length(sentence):
    words = sentence.split()
    total_word_length = 0
    word_count = 0
    for word in words:
        word = ''.join(c for c in word if c.isalnum())
        if word:
            total_word_length += len(word)
            word_count += 1
    if word_count > 0:
        average_length = total_word_length / word_count
        return average_length
    else:
        return 0

In [164]:
df['AVG WORD LENGTH'] = 0
df['AVG WORD LENGTH'] = df['text'].apply(calculate_average_word_length)
df

,URL_ID,URL,raw_text,text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 telem...,"[rise, telemedicine, impact, livelihood, 2040,...",74,21,0.557895,0.848214,9.551724,534,4.767857,5.727833,18.896552,8440,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",2,8.097473
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 rise e-health...,"[rise, e-health, impact, humans, 2030, rise, e...",38,13,0.490196,0.455357,11.680000,194,1.732143,5.364857,24.720000,2951,"{'rise': 2, 'e-health': 3, 'impact': 2, 'human...",3,8.113014
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,rise of e-health and its impact on humans by t...,rise e-health impact humans 2030 2020 world ra...,"[rise, e-health, impact, humans, 2030, 2020, w...",18,27,-0.200000,0.401786,7.148649,280,2.500000,3.859459,14.689189,5114,"{'rise': 2, 'e-health': 3, 'impact': 2, 'human...",3,7.450758
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068,6256,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",7,7.632148
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,rise of telemedicine and its impact on livelih...,rise telemedicine impact livelihood 2040 “more...,"[rise, telemedicine, impact, livelihood, 2040,...",35,27,0.129032,0.553571,11.000000,353,3.151786,5.660714,20.644068,6256,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",7,7.632148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,coronavirus: impact on the hospitality industr...,coronavirus: impact hospitality industry jumpi...,"[coronavirus, :, impact, hospitality, industry...",5,26,-0.677419,0.276786,10.242424,150,1.339286,4.632684,19.424242,3366,"{'coronavirus': 5, ':': 0, 'impact': 2, 'hospi...",1,7.664671
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,coronavirus impact on energy markets as the co...,coronavirus impact energy markets coronavirus ...,"[coronavirus, impact, energy, markets, coronav...",20,56,-0.473684,0.678571,15.980392,351,3.133929,7.645728,31.333333,8228,"{'coronavirus': 5, 'impact': 2, 'energy': 2, '...",3,7.387176
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,what are the key policies that will mitigate t...,policies mitigate impacts covid-19 world work?...,"[policies, mitigate, impacts, covid-19, world,...",80,32,0.428571,1.000000,12.597222,437,3.901786,6.599603,24.027778,8349,"{'policies': 3, 'mitigate': 4, 'impacts': 2, '...",0,7.270121
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,marketing drives results with a focus on probl...,marketing drives results focus problems whenth...,"[marketing, drives, results, focus, problems, ...",24,21,0.066667,0.401786,10.046875,337,3.008929,5.222321,19.625000,6397,"{'marketing': 3, 'drives': 1, 'results': 2, 'f...",7,7.908243


In [165]:
df.to_excel('Output Data Structure.xlsx')